# Human MPG-Intuition Pilot (CR-005)

This notebook sketches analysis steps linking MUFS, Potency, coherence, and human intuition self-report.

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

uri = 'bolt://localhost:7687'
user = 'neo4j'
password = 'neo4j-password'
driver = GraphDatabase.driver(uri, auth=(user, password))

In [ ]:
def fetch_dataframe(query, **params):
    with driver.session() as session:
        records = session.run(query, **params)
    rows = [r.data() for r in records]
    return pd.DataFrame(rows)

# MUFS incidence by awareness condition
df_mufs = fetch_dataframe(
    """
    MATCH (t:Trial)
    RETURN t.awareness_condition AS awareness,
           count(*) AS trials,
           sum(CASE WHEN t.mpg_intuitive THEN 1 ELSE 0 END) AS intuitive,
           sum(t.mufs_size) AS total_mufs_size
    """
)
df_mufs

In [ ]:
# Coherence vs intuition rating (requires SegmentState.coherence)
df_coh = fetch_dataframe(
    """
    MATCH (t:Trial)-[:HAS_MUFS]->(m:MUFS)
    OPTIONAL MATCH (t)-[:USES_SEGMENT]->(s:Segment)-[:HAS_STATE]->(st:SegmentState)
    WHERE st.coherence IS NOT NULL
    RETURN t.id AS trial_id, t.human_intuition_rating AS human_intuition,
           st.coherence AS coherence, st.potency AS potency, m.size AS mufs_size
    """
)
df_coh